In [1]:
import math
import pickle
import pandas as pd
from pprint import pprint
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
file_name_nl = '../data/excel/nl-dentsu-music-yt-ryan-matrix-102022.xlsx'
file_name_uk = '../data/excel/uk-dentsu-music-ryan-matrix-102822.xlsx'
df = pd.DataFrame(pd.read_excel(file_name_uk, skiprows = 1))
df.head()

,Unnamed: 0,Totals,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Sex (D2) [Female],Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows],Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Type Of Music/Formats You Regularly Listen To (L6) [Other],Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150
0,NaN,Unwgt,(000s),Vert%,Horz%,Index,Unwgt,(000s),Vert%,Horz%,...,Unwgt,(000s),Vert%,Horz%,Index,Unwgt,(000s),Vert%,Horz%,Index
1,Totals,10811,48441,100,100,100,5378,24294,100,50.15,...,1308,5973,100,12.33,100,814,3685,100,7.61,100
2,Actual Age (D1) [15 or 16 or 17 or 18 or 19 or...,1644,7640,15.77,100,100,788,3654,15.04,47.83,...,284,1300,21.76,17.01,138,138,686,18.61,8.98,118
3,Actual Age (D1) [25 or 26 or 27 or 28 or 29 or...,1844,8763,18.09,100,100,939,4335,17.84,49.47,...,320,1508,25.25,17.21,140,113,554,15.04,6.33,83
4,Actual Age (D1) [36 or 37 or 38 or 39 or 40 or...,1605,7245,14.96,100,100,779,3629,14.94,50.1,...,233,1048,17.55,14.47,117,71,326,8.83,4.49,59


In [3]:
def data_preprocess(file_name):
    df = pd.read_excel(file_name, skiprows = 1, header = [0, 1])
    df.columns = df.columns.map('_'.join)
    df = df.rename(columns = {'Unnamed: 0_level_0_Unnamed: 0_level_1': 'category'})
    df = df.set_index('category')
    df = df.rename(columns = lambda c: c.replace('Gender (D2) [Female]', 'female'))
    df = df.rename(columns = lambda c: c.replace('Gender (D2) [Male]', 'male'))
    return df

def cols_to_keep(df, starts_with, ends_with = ['Horz%', '(000s)']):
    cols_to_keep = []
    for col in list(df.columns):
        for start_with in starts_with:
            for end_with in ends_with:
                if (col.startswith(start_with) and col.endswith(end_with)):
                    cols_to_keep.append(col)
    
    return cols_to_keep

def rows_to_keep(df, starts_with):
    rows_to_keep = []
    for row in list(df.index.values):
        if not isinstance(row, str): continue
        for start_with in starts_with:
            if (row.startswith(start_with)):
                rows_to_keep.append(row)
    return rows_to_keep

def sub_df(df, cols_to_keep, rows_to_keep):
    df = df[cols_to_keep]
    df = df[df.index.isin(rows_to_keep)]
    return df

In [4]:
df = data_preprocess(file_name_nl)
df.head()

,Totals_Unwgt,Totals_(000s),Totals_Vert%,Totals_Horz%,Totals_Index,female_Unwgt,female_(000s),female_Vert%,female_Horz%,female_Index,...,Category vlog/YouTube follow (Q34) [DIY]_Unwgt,Category vlog/YouTube follow (Q34) [DIY]_(000s),Category vlog/YouTube follow (Q34) [DIY]_Vert%,Category vlog/YouTube follow (Q34) [DIY]_Horz%,Category vlog/YouTube follow (Q34) [DIY]_Index,Category vlog/YouTube follow (Q34) [Gadgets/technology]_Unwgt,Category vlog/YouTube follow (Q34) [Gadgets/technology]_(000s),Category vlog/YouTube follow (Q34) [Gadgets/technology]_Vert%,Category vlog/YouTube follow (Q34) [Gadgets/technology]_Horz%,Category vlog/YouTube follow (Q34) [Gadgets/technology]_Index
category,,,,,,,,,,,,,,,,,,,,,
Totals,5627.0,14274.0,100.00,100.0,100.0,2968,7184,100,50.33,100,...,663.0,1744.0,100.00,12.22,100.0,706.0,2013.0,100.00,14.10,100.0
Gender (D2) [Female],2968.0,7184.0,50.33,100.0,100.0,2968,7184,100,100,199,...,373.0,917.0,52.61,12.77,105.0,135.0,328.0,16.28,4.56,32.0
Gender (D2) [Male],2659.0,7090.0,49.67,100.0,100.0,-,-,-,-,-,...,290.0,826.0,47.39,11.66,95.0,571.0,1685.0,83.72,23.77,169.0
Actual age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or 24],564.0,1923.0,13.47,100.0,100.0,386,1113,15.5,57.89,115,...,127.0,418.0,23.97,21.73,178.0,102.0,427.0,21.19,22.18,157.0
Actual age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or 34],738.0,2325.0,16.29,100.0,100.0,402,1045,14.55,44.96,89,...,133.0,408.0,23.37,17.53,144.0,121.0,422.0,20.94,18.14,129.0


In [5]:
cols_to_keep_gender = cols_to_keep(df, ['Totals_(000s)', 'female', 'male'])
cols_to_keep_age = cols_to_keep(df, ['Totals_(000s)', 'Actual age (D1)'])

rows_to_keep = rows_to_keep(df, ['Preference music-Yes', 'Category vlog/YouTube follow'])

print('Columns for gender table')
pprint(cols_to_keep_gender)
print()
print('Columns for age table')
pprint(cols_to_keep_age)
print()
print('Rows for tables')
pprint(rows_to_keep)

Columns for gender table
['Totals_(000s)', 'female_(000s)', 'female_Horz%', 'male_(000s)', 'male_Horz%']

Columns for age table
['Totals_(000s)',
 'Actual age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or '
 '24]_(000s)',
 'Actual age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or '
 '24]_Horz%',
 'Actual age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or '
 '34]_(000s)',
 'Actual age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or '
 '34]_Horz%',
 'Actual age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or '
 '44]_(000s)',
 'Actual age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or '
 '44]_Horz%',
 'Actual age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or '
 '54]_(000s)',
 'Actual age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or '
 '54]_Horz%',
 'Actual age (D1) [55 or 56 or 57 or 58 or 59 or 60 or 61 or 62 or 63 or '
 '64]_(000s)',
 'Actual age (D1) [55 or 56 or 57 or 58 or 59 or

In [254]:
df_gender_cleaned = sub_df(df, cols_to_keep_gender, rows_to_keep)
df_age_cleaned = sub_df(df, cols_to_keep_age, rows_to_keep)

In [345]:
def get_padding(row, data_type):
    i, row_sum = 0, 0
    keys = set()
    for key in dict(row).keys():
        key = key.split('_')[0]
        keys.add(key) 
    keys.remove('Totals')
    for key in keys:
        row_sum += row[key + '_Horz%']
        i += 1
    return round((100.0 - row_sum) / i, 2)

# with open( '../data/mappings/music_all_mapping.pkl', 'rb') as f:
#     music_all_mapping = pickle.load(f)
# pprint(music_all_mapping) 
# print()
# with open( '../data/mappings/other_all_mapping.pkl', 'rb') as f:
#     other_all_mapping = pickle.load(f)
# pprint(other_all_mapping)    

In [346]:
def create_df_gender(df, country_code = 'UK', content_format_source_id = 3, geo_item_id = None):
    now = datetime.now()
    table = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'IsMale': [],
        'Percentage': [],
        'MaleCount': [],
        'FemaleCount': [],
        'TotalCount': []
    }
    
    with open( '../data/mappings/music_all_mapping.pkl', 'rb') as f:
        music_all_mapping = pickle.load(f)
    with open('../data/mappings/other_all_mapping.pkl', 'rb') as f:
        other_all_mapping = pickle.load(f)
    mappings = {**music_all_mapping, **other_all_mapping}
    
    for index, row in df.iterrows():
        mapping = mappings[index]
        
        table['ContentFormatId'].extend([mapping['id']] * 2)
        table['CategoryDB'].extend([mapping['table']] * 2)
        table['CategoryData'].extend([index] * 2)
        table['CountryCode'].extend([country_code] * 2)
        
        padding = get_padding(row, 'gender')
        table['IsMale'].append(False)
        table['Percentage'].append(row['female_Horz%'] + padding)
        table['FemaleCount'].append(row['female_(000s)'])
        table['MaleCount'].append(0)
        
        table['IsMale'].append(True)
        table['Percentage'].append(row['male_Horz%'] + padding)
        table['FemaleCount'].append(0)
        table['MaleCount'].append(row['male_(000s)'])
        
        table['TotalCount'].extend([row['male_(000s)'] + row['female_(000s)']] * 2)
    df = pd.DataFrame.from_dict(table)
    return df

In [347]:
df_gender_cleaned.head()

,Totals_(000s),female_(000s),female_Horz%,male_(000s),male_Horz%
category,,,,,
Preference music-Yes [Light classic],3488.0,1741,49.93,1747,50.07
Preference music-Yes [Heavy classic],1191.0,483,40.57,708,59.43
Preference music-Yes [Opera],662.0,309,46.69,353,53.31
Preference music-Yes [Dutch pop music],5590.0,2939,52.57,2651,47.43
Preference music-Yes [Dutch folk music],2698.0,1352,50.11,1346,49.89


In [348]:
df_gender_pre = create_df_gender(df_gender_cleaned, country_code = 'NL')
df_gender_pre.head()

,ContentFormatId,CategoryDB,CategoryData,CountryCode,IsMale,Percentage,MaleCount,FemaleCount,TotalCount
0,346,Classical Music,Preference music-Yes [Light classic],NL,False,49.93,0,1741,3488
1,346,Classical Music,Preference music-Yes [Light classic],NL,True,50.07,1747,0,3488
2,367,Hard Rock,Preference music-Yes [Heavy classic],NL,False,40.57,0,483,1191
3,367,Hard Rock,Preference music-Yes [Heavy classic],NL,True,59.43,708,0,1191
4,208,Opera,Preference music-Yes [Opera],NL,False,46.69,0,309,662


In [349]:
def get_row_info(row, min_age, max_age, padding):
    keys = set()
    for key in dict(row).keys():
        key = key.split('_')[0]
        keys.add(key)
    total = row['Totals_(000s)']
    for key in keys:
        if(str(min_age) in key or str(max_age) in key):
            percentage_with_padding = row[key + '_Horz%'] + padding
            return percentage_with_padding, round(total * percentage_with_padding / 100, 0)
    
def get_df_age(df, country_code = 'UK', content_format_source_id = 3, geo_item_id = None):
    now = datetime.now()
    table = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'MinAge': [],
        'MaxAge': [],
        'Percentage': [],
        'Count': [],
        'TotalCount': []
    }
    
    with open( '../data/mappings/music_all_mapping.pkl', 'rb') as f:
        music_all_mapping = pickle.load(f)
    with open('../data/mappings/other_all_mapping.pkl', 'rb') as f:
        other_all_mapping = pickle.load(f)
    mappings = {**music_all_mapping, **other_all_mapping}
    
    for index, row in df.iterrows():
        total_count = 0
        mapping = mappings[index]
        
        table['ContentFormatId'].extend([mapping['id']] * 6)
        table['CategoryDB'].extend([mapping['table']] * 6)
        table['CategoryData'].extend([index] * 6)
        table['CountryCode'].extend([country_code] * 6)
        
        padding = get_padding(row, 'age')
        
        table['MinAge'].append(0)
        table['MaxAge'].append(24)
        percentage, count = get_row_info(row, None, 24, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(25)
        table['MaxAge'].append(34)
        percentage, count = get_row_info(row, 25, 34, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(35)
        table['MaxAge'].append(44)
        percentage, count = get_row_info(row, 35, 44, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(45)
        table['MaxAge'].append(54)
        percentage, count = get_row_info(row, 45, 54, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(55)
        table['MaxAge'].append(64)
        percentage, count = get_row_info(row, 55, 64, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(65)
        table['MaxAge'].append(100)
        percentage, count = get_row_info(row, 65, None, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['TotalCount'].extend([total_count] * 6)
        
    df = pd.DataFrame.from_dict(table)
    return df

In [350]:
df_age_cleaned.head()

,Totals_(000s),Actual age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or 24]_(000s),Actual age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or 24]_Horz%,Actual age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or 34]_(000s),Actual age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or 34]_Horz%,Actual age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or 44]_(000s),Actual age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or 44]_Horz%,Actual age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or 54]_(000s),Actual age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or 54]_Horz%,Actual age (D1) [55 or 56 or 57 or 58 or 59 or 60 or 61 or 62 or 63 or 64]_(000s),Actual age (D1) [55 or 56 or 57 or 58 or 59 or 60 or 61 or 62 or 63 or 64]_Horz%,Actual age (D1) [65 or 66 or 67 or 68 or 69 or 70 or 71 or 72 or 73 or 74 or 75 or 76 or 77 or 78 or 79 or 80]_(000s),Actual age (D1) [65 or 66 or 67 or 68 or 69 or 70 or 71 or 72 or 73 or 74 or 75 or 76 or 77 or 78 or 79 or 80]_Horz%
category,,,,,,,,,,,,,
Preference music-Yes [Light classic],3488.0,239,6.84,374,10.72,324,9.3,443,12.69,796,22.81,1238,35.48
Preference music-Yes [Heavy classic],1191.0,110,9.2,138,11.59,107,9.02,111,9.29,269,22.59,409,34.39
Preference music-Yes [Opera],662.0,52,7.79,42,6.29,51,7.75,87,13.08,155,23.45,272,41.12
Preference music-Yes [Dutch pop music],5590.0,644,11.53,841,15.05,856,15.32,1094,19.57,1113,19.91,890,15.92
Preference music-Yes [Dutch folk music],2698.0,203,7.52,326,12.07,295,10.94,469,17.38,587,21.77,767,28.44


In [352]:
df_age_pre = get_df_age(df_age_cleaned, country_code = 'NL')
df_age_pre.head(12)

,ContentFormatId,CategoryDB,CategoryData,CountryCode,MinAge,MaxAge,Percentage,Count,TotalCount
0,346,Classical Music,Preference music-Yes [Light classic],NL,0,24,7.20,251.0,3487.0
1,346,Classical Music,Preference music-Yes [Light classic],NL,25,34,11.08,386.0,3487.0
2,346,Classical Music,Preference music-Yes [Light classic],NL,35,44,9.66,337.0,3487.0
3,346,Classical Music,Preference music-Yes [Light classic],NL,45,54,13.05,455.0,3487.0
4,346,Classical Music,Preference music-Yes [Light classic],NL,55,64,23.17,808.0,3487.0
5,346,Classical Music,Preference music-Yes [Light classic],NL,65,100,35.84,1250.0,3487.0
6,367,Hard Rock,Preference music-Yes [Heavy classic],NL,0,24,9.85,117.0,1190.0
7,367,Hard Rock,Preference music-Yes [Heavy classic],NL,25,34,12.24,146.0,1190.0
8,367,Hard Rock,Preference music-Yes [Heavy classic],NL,35,44,9.67,115.0,1190.0
9,367,Hard Rock,Preference music-Yes [Heavy classic],NL,45,54,9.94,118.0,1190.0


In [353]:
df_gender_pre.to_csv('../data/df_gender_pre_NL.csv')
df_age_pre.to_csv('../data/df_age_pre_NL.csv')

In [354]:
def unite_common_categories_gender(df, len_group = 2):
    common_content_id = []
    common_content = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'IsMale': [],
        'Percentage': [],
        'MaleCount': [],
        'FemaleCount': [],
        'TotalCount': []
    }
    content_groups = df.groupby('ContentFormatId')
    for content_id, content_group in content_groups:
        if (len(content_group) > len_group):
            common_content_id.append(content_id)
            
            common_content['ContentFormatId'].extend([content_id] * 2)
            common_content['CategoryDB'].extend([content_group['CategoryDB'].values[0]] * 2)
            common_content['CategoryData'].extend([content_group['CategoryData'].values[0]] * 2)
            common_content['CountryCode'].extend([content_group['CountryCode'].values[0]] * 2)
        
            content_sub_groups = content_group.groupby('IsMale')
            female_count = content_sub_groups['FemaleCount'].sum().values[0]
            male_count = content_sub_groups['MaleCount'].sum().values[1]
            total = content_sub_groups['TotalCount'].sum().values[1]
            
            common_content['IsMale'].append(False)
            common_content['Percentage'].append(round(female_count / total * 100, 2))
            common_content['FemaleCount'].append(female_count)
            common_content['MaleCount'].append(0)
            
            common_content['IsMale'].append(True)
            common_content['Percentage'].append(round(male_count / total * 100, 2))
            common_content['FemaleCount'].append(0)
            common_content['MaleCount'].append(male_count)
            
            common_content['TotalCount'].extend([total] * 2)
    
    df = df[~df['ContentFormatId'].isin(common_content_id)]
    df_new = pd.DataFrame.from_dict(common_content)
    df = pd.concat([df, df_new], ignore_index = True)
    df.to_csv('../data/df_gender_NL.csv')
    return df

def unite_common_categories_age(df, len_group = 6):
    common_content_id = []
    common_content = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'MinAge': [],
        'MaxAge': [],
        'Percentage': [],
        'Count': [],
        'TotalCount': []
    }
    
    content_groups = df.groupby('ContentFormatId')
    for content_id, content_group in content_groups:
        if (len(content_group) > len_group):
            common_content_id.append(content_id)
            
            common_content['ContentFormatId'].extend([content_id] * 6)
            common_content['CategoryDB'].extend([content_group['CategoryDB'].values[0]] * 6)
            common_content['CategoryData'].extend([content_group['CategoryData'].values[0]] * 6)
            common_content['CountryCode'].extend([content_group['CountryCode'].values[0]] * 6)
            
            content_sub_groups = content_group.groupby('MinAge', dropna = False)
            
            count_and_total = content_sub_groups[['Count', 'TotalCount']].sum().values

            common_content['MinAge'].append(0)
            common_content['MaxAge'].append(24)
            common_content['Percentage'].append(round(count_and_total[0][0] / count_and_total[0][1], 2))
            common_content['Count'].append(count_and_total[0][0])
            
            common_content['MinAge'].append(25)
            common_content['MaxAge'].append(34)
            common_content['Percentage'].append(round(count_and_total[1][0] / count_and_total[1][1], 2))
            common_content['Count'].append(count_and_total[1][0])
            
            common_content['MinAge'].append(35)
            common_content['MaxAge'].append(44)
            common_content['Percentage'].append(round(count_and_total[2][0] / count_and_total[2][1], 2))
            common_content['Count'].append(count_and_total[2][0])
            
            common_content['MinAge'].append(45)
            common_content['MaxAge'].append(54)
            common_content['Percentage'].append(round(count_and_total[3][0] / count_and_total[3][1], 2))
            common_content['Count'].append(count_and_total[3][0])
            
            common_content['MinAge'].append(55)
            common_content['MaxAge'].append(64)
            common_content['Percentage'].append(round(count_and_total[4][0] / count_and_total[4][1], 2))
            common_content['Count'].append(count_and_total[4][0])
            
            common_content['MinAge'].append(65)
            common_content['MaxAge'].append(100)
            common_content['Percentage'].append(round(count_and_total[5][0] / count_and_total[5][1], 2))
            common_content['Count'].append(count_and_total[5][0])
            
            common_content['TotalCount'].extend([count_and_total[0][1]] * 6)
            
    df = df[~df['ContentFormatId'].isin(common_content_id)]
    df_new = pd.DataFrame.from_dict(common_content)
    df = pd.concat([df, df_new], ignore_index = True)
    df.to_csv('../data/df_age_NL.csv')
    return df


In [355]:
df_age = unite_common_categories_age(df_age_pre)
df_age = df_age.drop(['CategoryDB', 'CategoryData', 'Count', 'TotalCount'], axis = 1)
df_age.to_csv('../data/table/df_age_NL_table.csv')
df_age

,ContentFormatId,CountryCode,MinAge,MaxAge,Percentage
0,346,NL,0,24,7.20
1,346,NL,25,34,11.08
2,346,NL,35,44,9.66
3,346,NL,45,54,13.05
4,346,NL,55,64,23.17
...,...,...,...,...,...
187,1021,NL,25,34,0.21
188,1021,NL,35,44,0.16
189,1021,NL,45,54,0.15
190,1021,NL,55,64,0.14


In [356]:
df_gender = unite_common_categories_gender(df_gender_pre)
df_gender = df_gender.drop(['CategoryDB', 'CategoryData', 'MaleCount', 'FemaleCount', 'TotalCount'], axis = 1)
df_gender.to_csv('../data/table/df_gender_NL_table.csv')
df_gender

,ContentFormatId,CountryCode,IsMale,Percentage
0,346,NL,False,49.93
1,346,NL,True,50.07
2,367,NL,False,40.57
3,367,NL,True,59.43
4,208,NL,False,46.69
...,...,...,...,...
59,354,NL,True,44.59
60,355,NL,False,51.44
61,355,NL,True,48.56
62,1021,NL,False,39.28
